# Q-Learning

Q-Learning은 강화학습의 대표적인 알고리즘 중 하나로, 환경과의 상호작용을 통해 최적의 정책을 학습하는 방법입니다.

<img src="https://www.researchgate.net/profile/Silvia-Ullo/publication/351884746/figure/fig1/AS:1067993664589824@1631640940747/Q-Learning-vs-Deep-Q-Learning.ppm" width="600">

[이미지 출처] https://www.researchgate.net/figure/Q-Learning-vs-Deep-Q-Learning_fig1_351884746

## 핵심 개념

- **Q-함수**: 상태(s)에서 행동(a)을 취했을 때의 예상 보상을 나타내는 함수
- **벨만 방정식**: Q(s,a) = r + γ * max(Q(s',a'))
  - r: 즉각적인 보상
  - γ: 할인 계수
  - s': 다음 상태
  - a': 다음 상태에서 가능한 모든 행동
- **ε-greedy 정책**: 탐험(exploration)과 활용(exploitation)의 균형을 위한 정책

## 알고리즘 단계

1. Q-테이블 초기화
2. 현재 상태 관찰
3. ε-greedy 정책에 따라 행동 선택
4. 행동 수행 및 보상 획득
5. Q-함수 업데이트
6. 새로운 상태로 이동
7. 2-6 단계 반복

## 장단점

### 장점:
- 모델 없이 학습 가능
- 수렴성 보장

### 단점:
- 큰 상태 공간에서는 비효율적
- 연속적인 상태나 행동 처리에 제한
  - 1. 메모리 요구량 증가 : 큰 상태 공간에서는 Q-테이블의 크기가 기하급수적으로 증가합니다. 각 상태-행동 쌍에 대한 Q-값을 저장해야 하므로, 상태의 수가 증가할수록 필요한 메모리가 급격히 늘어납니다.
  - 2. 학습 시간 증가 : 상태 공간이 커질수록 모든 상태-행동 쌍을 충분히 탐색하고 Q-값을 업데이트하는 데 필요한 시간이 크게 증가합니다. 이는 학습 속도를 현저히 저하시킵니다.
  - 3. 탐색-활용 딜레마 : 큰 상태 공간에서는 모든 상태를 충분히 탐색하기 어려워집니다. 이로 인해 최적 정책을 찾는 데 필요한 탐색과 학습된 정책을 활용하는 것 사이의 균형을 맞추기가 더욱 어려워집니다.

## 코드 설명

제공된 코드는 Q-Learning을 구현한 예제입니다. `Env` 클래스는 그리드 월드 환경을, `QLearningAgent` 클래스는 Q-Learning 에이전트를 구현합니다.

### 주요 함수 설명

1. `Env` 클래스:
   - `__init__`: 환경 초기화
   - `step`: 행동 수행 및 다음 상태, 보상 반환
   - `reset`: 환경 초기화

2. `QLearningAgent` 클래스:
   - `learn`: Q-함수 업데이트
   - `get_action`: ε-greedy 정책에 따른 행동 선택

### 코드 실행 과정

1. 환경과 에이전트 초기화
2. 에피소드 반복:
   - 상태 초기화
   - 행동 선택 및 수행
   - Q-함수 업데이트
   - 새로운 상태로 이동
   - Q-값 출력
3. 1000 에피소드 동안 반복

이 코드를 통해 에이전트는 그리드 월드에서 장애물을 피해 목표에 도달하는 최적 경로를 학습합니다.

* 아래 코드는 코랩 환경이 아닌 로컬 파이썬 환경에서 진행해주세요!
* 코드 출처 : https://github.com/rlcode/reinforcement-learning-kr

In [ ]:
# environment.py
import time
import numpy as np
import tkinter as tk
from PIL import ImageTk, Image

np.random.seed(1)
PhotoImage = ImageTk.PhotoImage
UNIT = 100  # 픽셀 수
HEIGHT = 5  # 그리드월드 세로
WIDTH = 5  # 그리드월드 가로


class Env(tk.Tk):
    def __init__(self):
        super(Env, self).__init__()
        self.action_space = ['u', 'd', 'l', 'r']
        self.n_actions = len(self.action_space)
        self.title('Q Learning')
        self.geometry('{0}x{1}'.format(HEIGHT * UNIT, HEIGHT * UNIT))
        self.shapes = self.load_images()
        self.canvas = self._build_canvas()
        self.texts = []

    def _build_canvas(self):
        canvas = tk.Canvas(self, bg='white',
                           height=HEIGHT * UNIT,
                           width=WIDTH * UNIT)
        # 그리드 생성
        for c in range(0, WIDTH * UNIT, UNIT):  # 0~400 by 80
            x0, y0, x1, y1 = c, 0, c, HEIGHT * UNIT
            canvas.create_line(x0, y0, x1, y1)
        for r in range(0, HEIGHT * UNIT, UNIT):  # 0~400 by 80
            x0, y0, x1, y1 = 0, r, HEIGHT * UNIT, r
            canvas.create_line(x0, y0, x1, y1)

        # 캔버스에 이미지 추가
        self.rectangle = canvas.create_image(50, 50, image=self.shapes[0])
        self.triangle1 = canvas.create_image(250, 150, image=self.shapes[1])
        self.triangle2 = canvas.create_image(150, 250, image=self.shapes[1])
        self.circle = canvas.create_image(250, 250, image=self.shapes[2])

        canvas.pack()

        return canvas

    def load_images(self):
        rectangle = PhotoImage(
            Image.open("../img/rectangle.png").resize((65, 65)))
        triangle = PhotoImage(
            Image.open("../img/triangle.png").resize((65, 65)))
        circle = PhotoImage(
            Image.open("../img/circle.png").resize((65, 65)))

        return rectangle, triangle, circle

    def text_value(self, row, col, contents, action, font='Helvetica', size=10,
                   style='normal', anchor="nw"):

        if action == 0:
            origin_x, origin_y = 7, 42
        elif action == 1:
            origin_x, origin_y = 85, 42
        elif action == 2:
            origin_x, origin_y = 42, 5
        else:
            origin_x, origin_y = 42, 77

        x, y = origin_y + (UNIT * col), origin_x + (UNIT * row)
        font = (font, str(size), style)
        text = self.canvas.create_text(x, y, fill="black", text=contents,
                                       font=font, anchor=anchor)
        return self.texts.append(text)

    def print_value_all(self, q_table):
        for i in self.texts:
            self.canvas.delete(i)
        self.texts.clear()
        for i in range(HEIGHT):
            for j in range(WIDTH):
                for action in range(0, 4):
                    state = [i, j]
                    if str(state) in q_table.keys():
                        temp = q_table[str(state)][action]
                        self.text_value(j, i, round(temp, 2), action)

    def coords_to_state(self, coords):
        x = int((coords[0] - 50) / 100)
        y = int((coords[1] - 50) / 100)
        return [x, y]

    def state_to_coords(self, state):
        x = int(state[0] * 100 + 50)
        y = int(state[1] * 100 + 50)
        return [x, y]

    def reset(self):
        self.update()
        time.sleep(0.5)
        x, y = self.canvas.coords(self.rectangle)
        self.canvas.move(self.rectangle, UNIT / 2 - x, UNIT / 2 - y)
        self.render()
        return self.coords_to_state(self.canvas.coords(self.rectangle))

    def step(self, action):
        state = self.canvas.coords(self.rectangle)
        base_action = np.array([0, 0])
        self.render()

        if action == 0:  # 상
            if state[1] > UNIT:
                base_action[1] -= UNIT
        elif action == 1:  # 하
            if state[1] < (HEIGHT - 1) * UNIT:
                base_action[1] += UNIT
        elif action == 2:  # 좌
            if state[0] > UNIT:
                base_action[0] -= UNIT
        elif action == 3:  # 우
            if state[0] < (WIDTH - 1) * UNIT:
                base_action[0] += UNIT

        # 에이전트 이동
        self.canvas.move(self.rectangle, base_action[0], base_action[1])
        # 에이전트(빨간 네모)를 가장 상위로 배치
        self.canvas.tag_raise(self.rectangle)
        next_state = self.canvas.coords(self.rectangle)

        # 보상 함수
        if next_state == self.canvas.coords(self.circle):
            reward = 100
            done = True
        elif next_state in [self.canvas.coords(self.triangle1),
                            self.canvas.coords(self.triangle2)]:
            reward = -100
            done = True
        else:
            reward = 0
            done = False

        next_state = self.coords_to_state(next_state)
        return next_state, reward, done

    def render(self):
        time.sleep(0.03)
        self.update()

In [ ]:
import numpy as np
import random
from environment import Env
from collections import defaultdict

class QLearningAgent:
    def __init__(self, actions):
        # 행동 = [0, 1, 2, 3] 순서대로 상, 하, 좌, 우
        self.actions = actions
        self.learning_rate = 0.01
        self.discount_factor = 0.9
        self.epsilon = 0.9
        self.q_table = defaultdict(lambda: [0.0, 0.0, 0.0, 0.0])

    # <s, a, r, s'> 샘플로부터 큐함수 업데이트
    def learn(self, state, action, reward, next_state):
        q_1 = self.q_table[state][action]
        # 벨만 최적 방정식을 사용한 큐함수의 업데이트
        q_2 = reward + self.discount_factor * max(self.q_table[next_state])
        self.q_table[state][action] += self.learning_rate * (q_2 - q_1)

    # 큐함수에 의거하여 입실론 탐욕 정책에 따라서 행동을 반환
    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            # 무작위 행동 반환
            action = np.random.choice(self.actions)
        else:
            # 큐함수에 따른 행동 반환
            state_action = self.q_table[state]
            action = self.arg_max(state_action)
        return action

    @staticmethod
    def arg_max(state_action):
        max_index_list = []
        max_value = state_action[0]
        for index, value in enumerate(state_action):
            if value > max_value:
                max_index_list.clear()
                max_value = value
                max_index_list.append(index)
            elif value == max_value:
                max_index_list.append(index)
        return random.choice(max_index_list)

if __name__ == "__main__":
    env = Env()
    agent = QLearningAgent(actions=list(range(env.n_actions)))

    for episode in range(1000):
        state = env.reset()

        while True:
            env.render()

            # 현재 상태에 대한 행동 선택
            action = agent.get_action(str(state))
            # 행동을 취한 후 다음 상태, 보상 에피소드의 종료여부를 받아옴
            next_state, reward, done = env.step(action)

            # <s,a,r,s'>로 큐함수를 업데이트
            agent.learn(str(state), action, reward, str(next_state))
            state = next_state
            # 모든 큐함수를 화면에 표시
            env.print_value_all(agent.q_table)

            if done:
                break